In [30]:
import numpy as np
import numba

DATA = (np.array([0, 0], dtype=np.float32), np.array([-3, 4], dtype=np.float32))


@numba.njit(parallel=True, fastmath=True, boundscheck=False, inline='always', nogil=True,)
def points_between_points(a: np.ndarray, b: np.ndarray):
    dist: float = np.linalg.norm(b-a)  # distance
    rise = (b[1]-a[1])/dist
    run:float = (b[0]-a[0])/dist
    return [(np.round(run*i), np.round(rise*i)) for i in range(np.ceil(dist)+1)]

def get_points_between_2_points(
    point_1: tuple[int, int], point_2: tuple[int, int]
) -> np.ndarray:
    """Return an array of coordinates of points that lie on the line between
    two given points.

    Args:
    -----
    point_1: A tuple of two integers that represent the (x, y) coordinates of
    the first point.

    point_2: A tuple of two integers that represent the (x, y) coordinates of
    the second point.

    Returns:
    -----
    np.ndarray: An array of coordinates of the points that lie on the line
    between the two given points.(inclusive of point1 and point2)

    Note:
    -----
    The function calculates the coordinates of the points that lie on the line
    between the two input points, and returns an array of these coordinates.
    The function first determines the slope and intercept of the line
    connecting the two input points. If the line is vertical, the function
    returns an array of points with the same x coordinate and a range of y
    coordinates. Otherwise, the function calculates the coordinates of the
    points on the line using the slope and intercept, and returns an array of
    these coordinates. The returned array is sorted by distance from the first
    input point.
    """

    x1, y1 = np.array(point_1).astype(int)
    x2, y2 = np.array(point_2).astype(int)

    # Calculate the slope of the line
    if x1 == x2:
        # Handle the special case where the line is vertical
        x_coords: np.ndarray = np.full(abs(y2 - y1) + 1, x1)
        y_coords: np.ndarray = np.arange(min(y1, y2), max(y1, y2) + 1)
    else:
        slope: int = (y2 - y1) // (x2 - x1)
        intercept: int = y1 - slope * x1

        # Calculate the coordinates of the points on the line
        x_coords: np.ndarray = np.arange(min(x1, x2), max(x1, x2) + 1)
        y_coords: np.ndarray = np.around(slope * x_coords + intercept)

    # Combine the x and y coordinates into a single array
    points: np.ndarray = np.column_stack((x_coords, y_coords))

    # Remove duplicate points
    points: np.ndarray = np.unique(points, axis=0)

    # Sort the points by distance from the first input point
    distances: np.ndarray = np.linalg.norm(points - point_1, axis=1)
    sorted_indices: np.ndarray = np.argsort(distances)
    points: np.ndarray = points[sorted_indices]

    return points.astype(int)

print(get_points_between_2_points(*DATA))

points_between_points(*DATA)  # ensure compilation

[[ 0  0]
 [-1  2]
 [-2  4]
 [-3  6]]


[(-0.0, 0.0), (-1.0, 1.0), (-1.0, 2.0), (-2.0, 2.0), (-2.0, 3.0), (-3.0, 4.0)]

In [31]:
%%timeit 

points_between_points(*DATA)

21.3 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
%%timeit

get_points_between_2_points(*DATA)

239 µs ± 30.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
